In [1]:
import os
from langchain_community.embeddings import OllamaEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredEPubLoader

from langchain_community.document_loaders import PyPDFLoader

embeddings = OllamaEmbeddings(model='nomic-embed-text:v1.5')

import logging

# setar logging como INFO


In [2]:
path_books = './books/import'
path_vector_store = '../vector-store/books'

In [3]:
def load_books(path_books):
    books = []
    for root, dirs, files in os.walk(path_books):
        for file in files:
            if file.endswith('.pdf'):
                books.append(os.path.join(root, file))
    return books

def load_pdf(file):
    loader = PyPDFLoader(file)
    documents = loader.load()
    return documents

def load_chunked_docs(documents):
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100
    )

    chunked_docs = text_spliter.split_documents(documents)

    return chunked_docs

def load_vector_store(chunked_docs, path_vector_store):
    if not os.path.isdir(path_vector_store):
        db = FAISS.from_documents(chunked_docs, embeddings)
        db.save_local(path_vector_store)
        print(db.index.ntotal)
        return db
    else:
        db = FAISS.load_local(folder_path=path_vector_store, 
                              embeddings=embeddings, 
                              allow_dangerous_deserialization=True)
        db.add_documents(chunked_docs)
        db.save_local(path_vector_store)      
        print(db.index.ntotal)
        return db

In [4]:
books = load_books(path_books)
documents_books = load_pdf(books[0])

In [5]:
chunked_docs = load_chunked_docs(documents_books)
db = load_vector_store(chunked_docs=chunked_docs, path_vector_store=path_vector_store)

3786


In [12]:
#testar se o documento foi adicionado
#db = FAISS.load_local("../vector-store/books", embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 3

results = retriever.get_relevant_documents("Calvinismo")


In [13]:
for row in results:
    page_content = row.page_content
    source = row.metadata['source']
    page = row.metadata['page']

    

    # Agora você pode fazer o que quiser com esses dados.
    # Por exemplo, você pode imprimi-los:
    print(f"""
Page Content: 
{page_content.replace('	',' ')}

Source: 
{source} 

Page:
{page}

""")


Page Content: 
um ministro para sua obra. Sempre digo aos jovens que me procuram a fim de que eu lhes dê
conselhos a respeito do ministério: “Não seja um ministro se puder evitar; pois, se o homem
pode evitar, é porque Deus não o chamou”. Entretanto, se ele não for capaz de evitar, se sentir
que precisa pregar ou morrerá, então é o homem que Deus quer. Que o Senhor o pressione, o
aperte e o constranja a pregar o evangelho, pois, se não houver uma compulsão divina, não
haverá compulsão espiritual em seu ministério sobre o coração dos que o ouvem. “Rogai ao
Senhor da colheita que mande trabalhadores para a sua colheita” (Mt 9.38).
Nosso Senhor disse: “para a sua colheita”. Eu gosto disso, porque a colheita não é nossa. Se essa
colheita perecer, é a colheita do nosso Pai celestial que perece. Isso põe um peso sobre a minha
alma. Se me dissessem que a colheita de algum tirano brutal e arrogante estava morrendo, eu
poderia dizer: “Que morra! Se ele a tivesse, que bem traria para ele ou qua

In [ ]:
loader = UnstructuredEPubLoader("/home/cristian/Documentos/Projetos/personal-search/ingestion/livros/Livro Conselhos para obreiros - Charles Spurgeon.epub", mode="elements")
data = loader.load()

documents = loader.load()